In [1]:
# -*- coding: utf-8 -*-
import logging
import sys

from gensim.corpora import WikiCorpus

读进所有文章

In [2]:
import os

direc = '/Users/Jesica/Documents/igolden/word2vec/word2vec-tutorial-master/news' # Get current working directory
ext = '.txt' # Select your file delimiter

file_dict = {} # Create an empty dict
txt = ''
# Select only files with the ext extension
txt_files = [i for i in os.listdir(direc) if os.path.splitext(i)[1] == ext]

# Iterate over your txt files
for f in txt_files:
    # Open them and assign them to file_dict
    with open(os.path.join(direc,f)) as file_object:
        text = file_object.read()
        file_dict[f] = text
        txt = txt + '\n'+ text


FileNotFoundError: [Errno 2] No such file or directory: '/Users/Jesica/Documents/igolden/word2vec/word2vec-tutorial-master/news'

句子分割

In [ ]:
def sentence_split(str_centence):
    list_ret = list()
    for s_str in str_centence.split('。'):
        if '?' in s_str:
            list_ret.extend(s_str.split('？'))
        elif '!' in s_str:
            list_ret.extend(s_str.split('！'))
        else:
            list_ret.append(s_str.strip())
    return list_ret

In [ ]:
l = sentence_split(txt)

In [ ]:
len(l)

句子分词

In [ ]:
import jieba

读取stopword

In [ ]:
stopwordset = set()
with open('jieba_dict/stopwords.txt','r',encoding = 'utf-8') as sw:
    for line in sw:
        stopwordset.add(line.strip('\n'))

In [ ]:
output = open('word_seg.txt','w')
texts_num = 0
for line in l:
    words = jieba.cut(line, cut_all = False)
    for word in words:
        if word not in stopwordset:
            output.write(word + ' ')
    texts_num += 1

output.close()

## Word2Vec 训练

In [ ]:

import math  
from string import punctuation  
from heapq import nlargest  
from itertools import product, count  
from gensim.models import word2vec  
import numpy as np  

In [ ]:
sentences = word2vec.Text8Corpus('word_seg.txt')
model = word2vec.Word2Vec(sentences, size =20)
model.save('w2v100.model.bin')

In [ ]:
#model2 = word2vec.Word2Vec(sentences, size=8, sg=1, hs=1, iter=10)
#model2.save('w2v_sg_5.model.bin')

In [ ]:
res = model.most_similar('保险',topn = 20)
for item in res:
    print(item[0]+","+str(item[1]))

In [ ]:
def cut_sentences(sentence):  
    puns = frozenset(u'。！？')  
    tmp = []  
    for ch in sentence:  
        tmp.append(ch)  
        if puns.__contains__(ch):  
            yield ''.join(tmp)  
            tmp = []  
    yield ''.join(tmp)

In [ ]:
def two_sentences_similarity(sents_1, sents_2):  
    ''''' 
    计算两个句子的相似性 
    相同词语的百分比
    :param sents_1: 
    :param sents_2: 
    :return: 
    '''  
    counter = 0  
    for sent in sents_1:  
        if sent in sents_2:  
            counter += 1  
    return counter / (math.log(len(sents_1) + len(sents_2)))  

In [ ]:
def create_graph(word_sent):  
    """ 
    传入句子链表  返回句子之间相似度的图 
    :param word_sent: 
    :return: 
    """  
    num = len(word_sent)  
    board = [[0.0 for _ in range(num)] for _ in range(num)]  
  
    for i, j in product(range(num), repeat=2):  
        if i != j:  
            
            board[i][j] = compute_similarity_by_avg(word_sent[i], word_sent[j])  
    return board  
  

In [ ]:
def cosine_similarity(vec1, vec2):  
    ''''' 
    计算两个向量之间的余弦相似度 
    :param vec1: 
    :param vec2: 
    :return: 
    '''  
    tx = np.array(vec1)  
    ty = np.array(vec2)  
    cos1 = np.sum(tx * ty)  
    cos21 = np.sqrt(sum(tx ** 2))  
    cos22 = np.sqrt(sum(ty ** 2))  
    cosine_value = cos1 / float(cos21 * cos22)  
    return cosine_value  

In [ ]:
def compute_similarity_by_avg(sents_1, sents_2):  
    ''''' 
    对两个句子求平均词向量 
    :param sents_1: 
    :param sents_2: 
    :return: 
    '''  
    if len(sents_1) == 0 or len(sents_2) == 0:  
        return 0.0  
    vec1 = model[sents_1[0]]  
    for word1 in sents_1[1:]:  
        vec1 = vec1 + model[word1]  
  
    vec2 = model[sents_2[0]]  
    for word2 in sents_2[1:]:  
        print
        vec2 = vec2 + model[word2]  
  
    similarity = cosine_similarity(vec1 / len(sents_1), vec2 / len(sents_2))  
    return similarity  
  

In [ ]:
def calculate_score(weight_graph, scores, i):  
    """ 
    计算句子在图中的分数 
    :param weight_graph: 
    :param scores: 
    :param i: 
    :return: 
    """  
    length = len(weight_graph)  
    d = 0.85  
    added_score = 0.0  
  
    for j in range(length):  
        fraction = 0.0  
        denominator = 0.0  
        # 计算分子  
        fraction = weight_graph[j][i] * scores[j]  
        # 计算分母  
        for k in range(length):  
            denominator += weight_graph[j][k]  
            if denominator == 0:  
                denominator = 1  
        added_score += fraction / denominator  
    # 算出最终的分数  
    weighted_score = (1 - d) + d * added_score  
    return weighted_score  

In [ ]:
def weight_sentences_rank(weight_graph):  
    ''''' 
    输入相似度的图（矩阵) 
    返回各个句子的分数 
    :param weight_graph: 
    :return: 
    '''  
    # 初始分数设置为0.5  
    scores = [0.5 for _ in range(len(weight_graph))]  
    old_scores = [0.0 for _ in range(len(weight_graph))]  
  
    # 开始迭代  
    while different(scores, old_scores):  
        for i in range(len(weight_graph)):  
            old_scores[i] = scores[i]  
        for i in range(len(weight_graph)):  
            scores[i] = calculate_score(weight_graph, scores, i)  
    return scores  

In [ ]:
def different(scores, old_scores):  
    ''''' 
    判断前后分数有无变化 
    :param scores: 
    :param old_scores: 
    :return: 
    '''  
    flag = False  
    for i in range(len(scores)):  
        if math.fabs(scores[i] - old_scores[i]) >= 0.0001:  
            flag = True  
            break  
    return flag 

In [ ]:
def filter_symbols(sents):  
    stopwords = list(stopwordset)+ ['。', ' ', '.',', ','印']  
    _sents = []  
    for sentence in sents:  
        _sent = []
        for word in sentence:  
            if word not in stopwordset:  
                _sent.append(word)
        if _sent:  
            _sents.append(_sent)  
    return _sents  

In [ ]:
def filter_model(sents):  
    _sents = []  
    for sentence in sents:  
        _sent = []
        for word in sentence:  
            if word in model: 
                _sent.append(word) 

        if _sent:  
            _sents.append(_sent)  
    return _sents  

In [ ]:
def summarize(text, n=0):  
    text = text.replace('\n','')
    tokens = cut_sentences(text) 
    s_count = 0
    sentences = []  
    sents = []  
    ratio = 0.1 #取文本10%
    for sent in tokens:  
        s_count = s_count + 1
        sentences.append(sent)  
        sents.append([word for word in jieba.cut(sent) if word ])  
#    print(sents)
    sents_fs = filter_symbols(sents)  

    sents_fm = filter_model(sents_fs)  
    graph = create_graph(sents_fm)  
    if n==0:
        n = math.floor(s_count*ratio)

    scores = weight_sentences_rank(graph)  
    sent_selected = nlargest(n, zip(scores, count()))  
    sent_index = []  

    for i in range(n):  
        sent_index.append(sent_selected[i][1])  
    #return [sentences[i] for i in sent_index]  
    return [(sentences[i]) for i in sent_index] 

In [ ]:
if __name__ == '__main__':  
    #from IPython.core.debugger import Tracer; Tracer()()
    with open("test/令人拍手叫好的年度幽默硬科幻巨作《火星救援》当中，主人公马特达蒙饰演的角色在火星上非常努力，种土豆养"
              , "r", encoding='utf-8') as myfile:  
        text = myfile.read().replace('\n', '')  
        
        print(summarize(text)) 
        print(summarize(text,5))